In [1]:
import tensorflow as tf

from tensorflow.keras.layers import Dense, concatenate, Activation,Dropout, Input, LSTM, Flatten, Embedding

from tensorflow.keras.models import Model

from tensorflow.keras.preprocessing.text import Tokenizer


from tensorflow.keras.preprocessing.sequence import pad_sequences
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
import pandas as pd
import numpy as np

import pandas as pd
import numpy as np

import fasttext
from gensim.utils import simple_preprocess
import csv
import fasttext
from sklearn.metrics import classification_report,accuracy_score
from gensim.models import KeyedVectors

In [2]:
data = pd.read_csv('train.txt', sep="__", header=None)
data[2] = data[2].apply(lambda x: '__label__' + x)
data.drop(1,axis=1,inplace=True)
data.rename(columns={0:'Consumer complaint narrative',2:'Target'},inplace=True)

C:\Users\suraj\anaconda3\envs\NLP\lib\site-packages\pandas\util\_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


In [9]:
df_sample = data

In [10]:
import re
import string
def clean_text(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    #text = " ".join(text)
    text = text.lower()
    text = text.strip()
    text = re.sub('\[.*?\]’', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = text.replace("’","")
    text = re.sub('[^A-Za-z0-9]+', ' ', text)
        
    return text

In [11]:
more_stp_words = ['xxxx', 'xxxxxxxx', 'report', 'reporting', 'loan', 
                  'company', 'debt', 'bank', 'received', 'accounts', 'sent', 'would', 'also', 'letter', 
                  'days', 'never', 'told','an','am','said','dont','like','dint','know','ive','got'
                 ,'could','see','havent','im','going','looked','through','tell','yeh',
                 'go','back','get','yer','would','never','seen','something','else','next','day','years'
                 ,'didnt','look','thousand','one','ever','even','though','although','every','time','make','sure'
                 ,'told','minutes','minute','hour','where','else','no','oh','ten','year','front','door','wandering','nine',
                 'ten','points','fifty','hundred','twenty','gotten','fell','asleep','yes','trying','find','one','two','three'
                 ,'four','thirteen','clock','give','us','youd','expect','picked','lived','number','twice','youknowwho','think',
                  'hed','harrys','well','around','items','due', 'please','sba','end','find','route','continues','look','cfpb',
                 'matter','look','said','good','tells','also','connected','help','business','email','working','many','work',
                 'still','want','talk','issues','issue','take','form','file','fair','consumer','reasonable','means','therefore',
                 'inquiries','cc','shall','call','date','open','attn','since','xx','credit','card','citi','citibank',
                 'account','called','made','information','balance']
stop_words.extend(more_stp_words)

In [12]:
def final_clean(texts):
    toks = word_tokenize(texts)
    stp = [word for word in toks if word not in stop_words]
    #stpr = ' '.join(stp)
    return stp

In [13]:
for i in range(len(df_sample)):
    df_sample['Consumer complaint narrative'][i] = clean_text(df_sample['Consumer complaint narrative'][i])

AttributeError: 'list' object has no attribute 'lower'

In [ ]:
for i in range(len(df_sample)):
    df_sample['Consumer complaint narrative'][i] = final_clean(df_sample['Consumer complaint narrative'][i])

In [ ]:
t = Tokenizer()
    
t.fit_on_texts(df_sample['Consumer complaint narrative'])

In [14]:
leng = []
for i in df_sample['Consumer complaint narrative']:
    leng.append(len(i))
    
import numpy as np
index_max = np.argmax(leng)
print(max(leng))
print(index_max)

1147
366


In [11]:
df_sample.reset_index(inplace=True,drop=True)

In [12]:
X_essay = t.texts_to_sequences(df_sample['Consumer complaint narrative'])

X_essay = pad_sequences(X_essay, maxlen=341, padding='pre')
# here 340 as 1 space counts 2 words

In [13]:
vocab_size = len(t.word_index) + 1


vocab_size

481

In [14]:
embeddings_index = dict()

f = open('/Users/akashbhoite/Downloads/glove.42B.300d.txt', encoding='utf8')

for line in f:

    values = line.split()

    word = values[0]

    coefs = np.asarray(values[1:])

    embeddings_index[word] = coefs

f.close()

print('Loaded %s word vectors.' % len(embeddings_index))#create a weight matrix for words in training docs

Loaded 1917495 word vectors.


In [17]:
embedding_matrix = np.zeros((vocab_size, 300))

for word, i in t.word_index.items():

    embedding_vector = embeddings_index.get(word)

    if embedding_vector is not None:

        embedding_matrix[i] = embedding_vector

In [ ]:
model = fasttext.train_supervised('train.txt',pretrainedVectors=embeddings_index,wordNgrams = 2)

In [ ]:
model.test('test.txt')      

In [ ]:
predictions = np.array(predictions[0]).flatten()
target = df_sample['Target'].to_numpy()

In [ ]:
print(classification_report(target,predictions))

In [ ]:
accuracy_score(target,predictions)